## Кривоногов Н.В., NLP, практическое задание № 16 - КУРСОВАЯ РАБОТА

### Создание чат-бота с тремя интентами: общение, покупки, уточнение запроса / непонимание запроса.

Ссылка на данный ноутбук в Colab:

https://colab.research.google.com/drive/1xlabzTrza7ULRp_YND4_dKwyA52SJa3-?usp=sharing

In [1]:
# # Проверка версий установленных библиотек:

# pip freeze

Установка и импорт библиотек:

In [1]:
!pip install python-telegram-bot==13.3

In [2]:
!pip install pymorphy2

In [3]:
!pip install stop_words

In [4]:
!pip install annoy

In [5]:
!pip install pandarallel

In [6]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import pickle

tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
from tqdm import tqdm, tqdm_notebook

In [8]:
#.   RESTART KERNEL

In [9]:
import mmap
import re

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

Требуется скачать файл Otvety.txt по ссылке (1,72G):

https://drive.google.com/file/d/1DQL9ybca4USImUDaxxHmkIZNmClKBtKG/view

In [11]:
!gdown 1DQL9ybca4USImUDaxxHmkIZNmClKBtKG

Downloading...
From: https://drive.google.com/uc?id=1DQL9ybca4USImUDaxxHmkIZNmClKBtKG
To: /content/Otvety.txt
100% 1.85G/1.85G [00:31<00:00, 58.7MB/s]


In [12]:
# Преобразование скачанного файла вопросов-ответов в строчный вид:

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("/content/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-12-57a1df5c5b8e>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [10]:
# Препроцессинг текста:

def preprocess_txt(line):

    line = re.sub('<[^<]+?>', '', line)
    line = re.sub(r'http\S+', '', line)
    line = re.sub('[^а-яА-ЯёЁ0-9\s]', '', line)
    line = re.sub(r'[^\w\s]', '', line)

    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]

    return spls

In [15]:
# Обработка текста:

sentences = []
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

file_path_from = 'prepared_answers.txt'
file_path_to = 'Otvety2.txt'

N = get_num_lines(file_path_from)
with open(file_path_to, mode = 'w') as fileto:
    with open(file_path_from) as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            spls = preprocess_txt(line)
            sentences.append(spls)
            c += 1
            if c > 500000000:
                break
            fileto.write(' '.join(spls)+'\n')
filefrom.close()
fileto.close()

100%|██████████| 1163342/1163342 [6:19:09<00:00, 51.14it/s]


Ссылка на уже подготовленный файл Otvety2.txt:

https://drive.google.com/file/d/1xP-DU3qJTTUP12nAt64YGyeltQ0bss7F/view?usp=sharing

In [11]:
# !gdown 1xP-DU3qJTTUP12nAt64YGyeltQ0bss7F

Downloading...
From: https://drive.google.com/uc?id=1xP-DU3qJTTUP12nAt64YGyeltQ0bss7F
To: /content/Otvety2.zip
100% 186M/186M [00:03<00:00, 50.2MB/s]


In [12]:
# !unzip '/content/Otvety2.zip'

Archive:  /content/Otvety2.zip
replace Otvety2.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Otvety2.txt             


In [13]:
# Загрузка результата:

sentences = []

file_path_from = '/content/Otvety2.txt'
#file_path_to = 'Otvety2.txt'

N = get_num_lines(file_path_from)
with open(file_path_to, mode = 'r') as filefrom:
    for k in tqdm(range(N)):
        line = filefrom.readline()
        if line == '': break
        sentences.append(line.split())
filefrom.close()

100%|██████████| 1163342/1163342 [00:25<00:00, 45167.87it/s]


In [14]:
vec = []
_ = [vec.extend(x)  for x in sentences[:100]]
vec = list(set(vec))
vec.sort()
vec[100:110]

['активный',
 'актуальный',
 'актёр',
 'алгоритм',
 'аллизантин',
 'аллисата',
 'аллицин',
 'альбом',
 'американский',
 'аммиак']

In [15]:
sentences[:10]

[['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой',
  'вобщий',
  'прикалывать',
  'тема'],
 ['делать',
  'найти',
  '2',
  'миллион',
  'рубль',
  'счастие',
  'свалиться',
  'хороший',
  'пойти',
  'милиция',
  'заявить',
  'находка',
  'деньга',
  'тероть',
  'самый',
  'интересный',
  'неприменный',
  'искать',
  'поверьте',
  'найти',
  'видеть',
  'подобный',
  'нарваться',
  'бабушка',
  'помочий',
  'внук',
  'покупка',
  'квартира',
  'бандит',
  'разговаривать',
  'иначе',
  'бабушка',
  'милиция',
  'выбор',
  'шанс',
  'подарок',
  'выше',
  'котрый',
  'никто',
  'спросить',
  'хороший',
  'отдать',
  'хотяб',
  '500',
  'благотворительность',
  'дабы',
  'спугнуть',
  'удача'],
 ['эбу',
  'двенашка',
  'называться',
  'итэлма',
  'эбу',
  'эбу',
  'электронный',
  'блок',
  'управление',
  'двигатель',
  'автомобиль',
  'название',
  'контроллер',
  'принимат

In [16]:
len(sentences)

1163342

In [17]:
sentences = sentences[:500000]

In [18]:
# len(sentences)

In [19]:
# Обучение модели FastText:

file_path_from = 'ft_model'

sentences = [i for i in tqdm(sentences) if len(i) > 2]
modelFT = FastText(sentences=sentences, vector_size=100, min_count=1, window=5)
modelFT.save("ft_model")

100%|██████████| 500000/500000 [00:01<00:00, 377234.94it/s]


In [25]:
# Загрузка модели:

# modelFT = FastText.load("ft_model")
# ft_index = annoy.AnnoyIndex(100 ,'angular')

In [26]:
#Размерность вектора модели. Если установить 100 — каждое слово в корпусе будет представлено в виде 100-мерного вектора и т.п.
#Наименьшее допустимое количество символов в слове, для которого будет создаваться векторное представление: так можно убрать частотные, но не очень значимые слова, типа союзов и предлогов.
#Размер окна. Этот параметр задает, сколько соседних слов считается частью контекста. Если выставить 40, то алгоритм возьмет 40 слов спереди от слова и 40 слов сзади от слова.

In [20]:
list(set(get_stop_words("ru")))[:20]

['своей',
 'ними',
 'одиннадцатый',
 'шесть',
 'он',
 'свое',
 'то',
 'твой',
 'просто',
 'вся',
 'мною',
 'нем',
 'вот',
 'саму',
 'недавно',
 'довольно',
 'никогда',
 'стал',
 'раньше',
 'наиболее']

In [22]:
# Создание индексов для вопросов-ответов:

file_path_from = 'speaker.ann'

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("Otvety2.txt", "r") as f:
    for line in tqdm(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[0]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)

        counter += 1

        if counter > 50_000:
            break

ft_index.build(10)
ft_index.save('speaker.ann')

with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

50000it [07:22, 112.88it/s]


In [23]:
#  Загрузка индексов:

ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')
index_map = pd.read_pickle("index_speaker.pkl")

Требуется скачать файл ProductsDataset.csv по ссылке (16M):

https://gbcdn.mrgcdn.ru/uploads/asset/5209459/attachment/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv

In [25]:
!gdown https://gbcdn.mrgcdn.ru/uploads/asset/5209459/attachment/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv

Downloading...
From: https://gbcdn.mrgcdn.ru/uploads/asset/5209459/attachment/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv
To: /content/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv
100% 16.4M/16.4M [00:06<00:00, 2.42MB/s]


In [26]:
# Создание модели продуктовых данных:

shop_data = pd.read_csv("/content/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv")
#shop_data = shop_data.iloc[:5000, :]

shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].progress_apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, новый, носить, реал, красивый,..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [27]:
# Подготовка для создания модели по определению домена данных:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [28]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))

# Вопрос-ответный домен:
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in tqdm(idxs)]

# Продуктовый домен:
positive_texts = [" ".join(val) for val in tqdm(shop_data['text'].values)]

100%|██████████| 35548/35548 [00:00<00:00, 621034.14it/s]


In [29]:
# ВО = 0, Прод = 1:

dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2,
                                                    stratify=labels, random_state=13)

In [31]:
# Модель:

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [32]:
# Качество:

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9723793131710515

Метрика достаточно высокая: модель должна хорошо распознавать и разделять домены (интенты).

In [33]:
# Добавление IDF взвешивания (для каждого слова поиск IDF веса):

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [35]:
np.mean(tfidf_vect.idf_)

10.691477561067922

In [36]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [40]:
# Создание индексов для продуктовых данных:

file_path_from = 'shop.ann'

ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

file_path_from = 'index_shop.pkl'
if not os.path.isfile(file_path_from):

    with open("index_shop.pkl", "wb") as f:
        pickle.dump(index_map_shop, f)


100%|██████████| 35548/35548 [00:09<00:00, 3938.84it/s]


In [41]:
# Загрузка индексов:

midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann')

index_map_shop = pd.read_pickle("index_shop.pkl")

In [42]:
# Основная функция преобразования текста в вектор х100:

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [44]:
updater = Updater("6136908859:AAE3dLIWPojlz-Hhq7Pasjdj3qXykohOeJw", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, я бот talk_and_buy. Я могу определить, что ты хочешь: поболтать или что-то купить. Давай пообщаемся?')

def textMessage(update, context):

    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)

    # ПРОД
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return

    # QA
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)

    #
    if distances[0] <=0.9: #> 100.5: #<=0.9:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Не понимаю... Спроси по-другому.")
        return

    # Вопрос-Ответ
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

0.8922566175460815
0.514726459980011


<ipython-input-42-0363a892ecac>:10: RuntimeWarning: invalid value encountered in divide
  return vector_ft / n_ft


0.514726459980011
0.514726459980011


https://t.me/talk_and_buy_bot